# **MS1 calibration for shear fatigue behavior - not included at Euro-C**

Study not included at Euro-C. The notebook can be reused for calibration

In [1]:
%matplotlib widget
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import matplotlib.pylab as plt
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.mats3D import MATS3DElastic
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.sim.tstep_bc import TStepBC
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from mayavi import mlab
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.tfunction import TimeFunction, TFSelector, TFCyclicSin, TFCyclicNonsymmetricConstant
import copy

## Unit element Axisymm 

In [3]:
n_x_e = 3
n_inner_y_e = 1
n_outer_y_e = 1
L_x = 90.0 # [mm]
d_x = L_x / n_x_e
R_in = 25 # [mm]
dR = 4 # [mm]
R_out = 50 # [mm]
len_notch = 30
n_notch = 1

xmodel = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0+n_notch*d_x, R_in),
                               coord_max=(L_x-n_notch*d_x, R_in+dR),
                               shape=(n_x_e-2*n_notch, 1),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=40000, nu=0.18)
m = TStepBC(
    domains=[(xmodel, m_elastic),
             ]
)

In [4]:
tf_precrompression = TFBilinear(loading_ratio = 1.0, time_ratio=0.1)
tf_sliding = TFBilinear(loading_ratio = 0.0, time_ratio=0.1)

In [5]:
slide_upper_slice = BCSliceI(slice=xmodel.mesh.I[:, -1],
                                   var='u', dims=[0], value=0.0)
slide_upper_dofs = slide_upper_slice.dofs

slide_lower_slice = BCSliceI(slice=xmodel.mesh.I[:, 0],
                                   var='u', dims=[0], value=0.0)
slide_lower_dofs = slide_lower_slice.dofs

confinement_upper_slice = BCSliceI(slice=xmodel.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.0)
confinement_upper_dofs = confinement_upper_slice.dofs

confinement_lower_slice = BCSliceI(slice=xmodel.mesh.I[:, 0],
                                   var='u', dims=[1], value=0.0)
confinement_lower_dofs = confinement_lower_slice.dofs


## Monotonic - dc

In [8]:
compression_stress = -30
slide = 0.1


m = TStepBC(
domains=[(xmodel, m_elastic),
         ]
)
# Shear bc

inner_fixed_0 = slide_lower_slice
inner_fixed_1 = confinement_lower_slice

compression_force = compression_stress * 2 * np.pi * (R_in + dR) * len_notch  / (len(confinement_upper_dofs))
upper_compression_force = [BCDof(var='f', dof=dof, value=compression_force, 
                                 time_function=tf_precrompression)
                           for dof in confinement_upper_dofs]

slide_upper = [BCDof(var='u', dof=dof, value=slide, time_function=tf_sliding)
                     for dof in slide_upper_dofs]

bc1 = [inner_fixed_0, inner_fixed_1] + slide_upper + upper_compression_force

m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = False
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains
s.reset()
s.run()

In [13]:
cm = 1/2.54  # centimeters in inches
# plt.style.use('grayscale')
plt.style.use('default')

plt.tight_layout()
max_F = []
plt.rcParams["font.family"] = "Times New Roman"

fig, ((ax1,ax2,ax3)) = plt.subplots(1, 3,figsize=(21*cm, 7*cm))

U_slide = np.average(m.hist.U_t[:, slide_upper_dofs], axis=-1)
U_dilatancy  =m.hist.U_t[:, confinement_upper_dofs[0]]
F_slide = np.sum(m.hist.F_t[:, slide_upper_dofs], axis=-1)

F_confinement = np.sum(m.hist.F_t[:, confinement_upper_dofs], axis=-1)

ax1.plot(U_slide, F_confinement/(2 * np.pi * (R_in+dR)*(L_x-n_notch*2*d_x)))
ax1.set_ylabel('confinement [MPa]', fontsize=10)
ax1.set_xlabel('slip [mm]', fontsize=10)

ax2.plot(U_slide, F_slide/1000)
ax2.set_ylabel('shear force [kN]', fontsize=10)
ax2.set_xlabel('slip [mm]', fontsize=10)

ax3.plot(U_slide, U_dilatancy)
ax3.set_ylabel('dilatancy [mm]', fontsize=10)
ax3.set_xlabel('slip [mm]', fontsize=10)
    
    
left  = 0.1  # the left side of the subplots of the figure
right = 0.98  # the right side of the subplots of the figure
bottom = 0.2   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.4   # the amount of width reserved for blank space between subplots
hspace = 0.4   # the amount of height reserved for white space between subplots
# mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b"]) 
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=None, wspace=wspace, hspace=hspace)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Monotonic - fc

In [14]:
compression_stress = -30
slide = 2000 * 1000


m = TStepBC(
domains=[(xmodel, m_elastic),
         ]
)
# Shear bc

inner_fixed_0 = slide_lower_slice
inner_fixed_1 = confinement_lower_slice

compression_force = compression_stress * 2 * np.pi * (R_in + dR) * len_notch  / (len(confinement_upper_dofs))
upper_compression_force = [BCDof(var='f', dof=dof, value=compression_force, 
                                 time_function=tf_precrompression)
                           for dof in confinement_upper_dofs]

slide_upper = [BCDof(var='f', dof=dof, value=slide, time_function=tf_sliding)
                     for dof in slide_upper_dofs]

bc1 = [inner_fixed_0, inner_fixed_1] + slide_upper + upper_compression_force

m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = False
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains
s.reset()
s.run()

In [15]:
cm = 1/2.54  # centimeters in inches
# plt.style.use('grayscale')
plt.style.use('default')

plt.tight_layout()
max_F = []
plt.rcParams["font.family"] = "Times New Roman"

fig, ((ax1,ax2,ax3)) = plt.subplots(1, 3,figsize=(21*cm, 7*cm))

U_slide = np.average(m.hist.U_t[:, slide_upper_dofs], axis=-1)
U_dilatancy  =m.hist.U_t[:, confinement_upper_dofs[0]]
F_slide = np.sum(m.hist.F_t[:, slide_upper_dofs], axis=-1)

F_confinement = np.sum(m.hist.F_t[:, confinement_upper_dofs], axis=-1)

ax1.plot(U_slide, F_confinement/(2 * np.pi * (R_in+dR)*(L_x-n_notch*2*d_x)))
ax1.set_ylabel('confinement [MPa]', fontsize=10)
ax1.set_xlabel('slip [mm]', fontsize=10)

ax2.plot(U_slide, F_slide/1000)
ax2.set_ylabel('shear force [kN]', fontsize=10)
ax2.set_xlabel('slip [mm]', fontsize=10)

ax3.plot(U_slide, U_dilatancy)
ax3.set_ylabel('dilatancy [mm]', fontsize=10)
ax3.set_xlabel('slip [mm]', fontsize=10)
    
    
left  = 0.1  # the left side of the subplots of the figure
right = 0.98  # the right side of the subplots of the figure
bottom = 0.2   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.4   # the amount of width reserved for blank space between subplots
hspace = 0.4   # the amount of height reserved for white space between subplots
# mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b"]) 
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=None, wspace=wspace, hspace=hspace)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …